In [ ]:
"""
To look at the status of our limb processing
"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload
import numpy_utils as nu
import trimesh_utils as tu
import error_detection as ed
import networkx as nx
import networkx_utils as xu
import neuron_utils as nru
import neuron_visualizations as nviz

INFO - 2020-12-10 21:21:43,170 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-10 21:21:43,171 - settings - Setting database.user to celiib
INFO - 2020-12-10 21:21:43,172 - settings - Setting database.password to newceliipass
INFO - 2020-12-10 21:21:43,176 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-10 21:21:43,177 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-10 21:21:43,190 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-10 21:21:43,407 - settings - Setting enable_python_native_blobs to True


In [31]:
minnie.LimbStats()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),limb_idx the limb id path was taken from,path_idx path identifier,soma_angle,n0_width_median_mesh_center,n0_width_no_spine_median_mesh_center,n0_n_spines,n0_total_spine_volume,n0_spine_volume_median,n0_spine_volume_density,n0_skeletal_length,n0_parent_angle,n0_sibling_angle,n1_width_median_mesh_center,n1_width_no_spine_median_mesh_center,n1_n_spines,n1_total_spine_volume,n1_spine_volume_median,n1_spine_volume_density,n1_skeletal_length,n1_parent_angle,n1_sibling_angle
864691134884740858,0,0.25,0,0,155.81,192.16,192.16,0,0.0,0.0,0.0,4891.73,28.43,140.83,328.27,327.31,4,184622195.49,50054356.2,2612.59,70666.48,13.53,96.8
864691134884740858,0,0.25,0,1,155.81,192.16,192.16,0,0.0,0.0,0.0,4891.73,28.43,140.83,99.67,99.67,0,0.0,0.0,0.0,12721.84,84.41,96.8
864691134884740858,0,0.25,0,2,155.81,83.79,83.79,0,0.0,0.0,0.0,6600.36,113.07,140.83,86.33,86.33,0,0.0,0.0,0.0,5910.65,29.57,31.68
864691134884740858,0,0.25,0,3,155.81,83.79,83.79,0,0.0,0.0,0.0,6600.36,113.07,140.83,63.25,63.25,0,0.0,0.0,0.0,6464.83,4.32,31.68
864691134884740858,0,0.25,1,0,36.37,363.8,363.8,0,0.0,0.0,0.0,9932.58,36.37,54.68,397.66,317.97,44,2416439973.27,42964302.31,46091.42,52427.12,18.83,52.52
864691134884740858,0,0.25,1,1,36.37,363.8,363.8,0,0.0,0.0,0.0,9932.58,36.37,54.68,356.78,290.76,79,3484836219.52,17590581.4,31853.38,109402.38,37.89,52.52
864691134884740858,0,0.25,1,2,36.37,381.7,377.51,3,214808934.88,46665548.02,20683.44,10385.55,18.78,54.68,360.14,332.93,11,752869412.34,32461787.3,31361.65,24006.05,35.5,52.65
864691134884740858,0,0.25,1,3,36.37,381.7,377.51,3,214808934.88,46665548.02,20683.44,10385.55,18.78,54.68,293.0,274.23,10,910711396.61,89051017.84,33856.04,26899.52,21.63,52.65
864691134884740858,0,0.25,2,0,135.85,398.13,398.13,2,22406048.94,11203024.47,3238.06,6919.6,53.41,99.67,333.97,280.57,66,2608981593.01,19915909.02,22816.62,114345.65,21.51,66.94
864691134884740858,0,0.25,2,1,135.85,398.13,398.13,2,22406048.94,11203024.47,3238.06,6919.6,53.41,99.67,282.86,252.85,26,830065912.26,25768129.64,21588.26,38449.87,49.41,66.94


In [ ]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

In [3]:
len(minnie.LimbStats())

278318

In [4]:
len(minnie.Decomposition.proj() & minnie.LimbStats.proj())

17073

# Downloading the Data

In [ ]:
import time
download_start_time = time.time()
total_data_dict = minnie.LimbStats.fetch(as_dict=True)
print(f"Total time for download = {time.time() - download_start_time}")

# Convert to a pandas table

In [ ]:
import pandas as pd
import numpy as np
path_df = pd.DataFrame.from_dict(total_data_dict)

In [ ]:
path_df.to_pickle("neuron_dendrite_path_data")
df_recov = pd.read_pickle("neuron_dendrite_path_data")
df_recov

# Saving the Data

In [ ]:
import system_utils as su
su.decompress_pickle(total_data_dict,"")

# Seeing If Can Get approximately labeled data by the angle (spot checking neurons)

In [6]:
angle_interval = [0,20]
restrict_table = (minnie.LimbStats() & f"soma_angle>={angle_interval[0]} AND soma_angle<={angle_interval[-1]}")
print(f"Length of restricted table = {len(restrict_table)}")
curr_data = restrict_table.fetch("segment_id","limb_idx")

Length of restricted table = 25480


In [7]:
restrict_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),limb_idx the limb id path was taken from,path_idx path identifier,soma_angle,n0_width_median_mesh_center,n0_width_no_spine_median_mesh_center,n0_n_spines,n0_total_spine_volume,n0_spine_volume_median,n0_spine_volume_density,n0_skeletal_length,n0_parent_angle,n0_sibling_angle,n1_width_median_mesh_center,n1_width_no_spine_median_mesh_center,n1_n_spines,n1_total_spine_volume,n1_spine_volume_median,n1_spine_volume_density,n1_skeletal_length,n1_parent_angle,n1_sibling_angle
864691134884741114,0,0.25,0,0,14.12,360.81,315.02,31,1279794941.1,19918144.63,32172.72,39778.88,38.53,69.61,459.64,338.79,42,2078597204.11,29460801.27,53434.5,38899.91,9.86,68.74
864691134884741114,0,0.25,0,1,14.12,360.81,315.02,31,1279794941.1,19918144.63,32172.72,39778.88,38.53,69.61,78.79,78.79,0,0.0,0.0,0.0,1648.23,73.25,68.74
864691134884741114,0,0.25,0,2,14.12,507.09,502.62,1,68994638.81,68994638.81,7810.44,8833.65,31.09,69.61,493.02,359.18,43,2147909537.53,37494720.13,60931.54,35251.19,83.73,99.98
864691134884741114,0,0.25,0,3,14.12,507.09,502.62,1,68994638.81,68994638.81,7810.44,8833.65,31.09,69.61,465.7,455.04,3,239821084.84,47691454.33,30452.73,7875.19,20.51,99.98
864691134884743418,0,0.25,0,0,13.79,463.15,463.15,2,170874075.01,85437037.51,15547.33,10990.57,83.78,107.32,299.95,297.23,6,311129649.71,53287623.74,29206.54,10652.74,11.11,66.76
864691134884743418,0,0.25,0,1,13.79,463.15,463.15,2,170874075.01,85437037.51,15547.33,10990.57,83.78,107.32,389.81,311.76,45,2317727451.9,32725447.82,39667.92,58428.25,64.79,66.76
864691134884743418,0,0.25,0,2,13.79,523.06,501.91,12,1724471104.36,145828608.97,55717.27,30950.39,23.56,107.32,619.22,470.76,40,3851479990.58,96536711.56,108958.19,35348.24,28.27,80.29
864691134884743418,0,0.25,0,3,13.79,523.06,501.91,12,1724471104.36,145828608.97,55717.27,30950.39,23.56,107.32,253.96,215.99,7,449241265.95,53784489.12,36589.86,12277.75,57.39,80.29
864691134884744442,0,0.25,0,0,19.68,268.02,268.02,0,0.0,0.0,0.0,1234.6,24.34,96.47,334.02,334.02,0,0.0,0.0,0.0,2611.29,41.59,71.82
864691134884744442,0,0.25,0,1,19.68,268.02,268.02,0,0.0,0.0,0.0,1234.6,24.34,96.47,434.95,434.95,7,226240228.64,10853983.38,5573.17,40594.56,30.3,71.82


In [8]:
unique_seg_id_branch_idx = np.unique(np.array(curr_data).T,axis=0)
unique_seg_id_branch_idx

array([[864691134884741114,                  0],
       [864691134884743418,                  0],
       [864691134884744442,                  0],
       ...,
       [864691137196926273,                  1],
       [864691137196928321,                  0],
       [864691137196928577,                  1]])

In [9]:
correct_apical_classification = []
wrong_apical_classification = []

In [ ]:
correct_apical_classification

# Looking at an example branch

In [28]:
idx = 6
curr_segment_id,curr_limb_idx = unique_seg_id_branch_idx[idx]
curr_segment_id,curr_limb_idx

(864691134884753658, 2)

In [29]:
import neuron_visualizations as nviz
neuron_obj = (minnie.Decomposition & dict(segment_id=curr_segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [30]:
nviz = reload(nviz)
print(f"Apical: Segment_id:{curr_segment_id}, Limb: {curr_limb_idx}")
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh"],
                     limb_branch_dict={f"L{curr_limb_idx}":"all"},
                     mesh_resolution="limb",
                     mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True,
                     verbose=False)

Apical: Segment_id:864691134884753658, Limb: 2


In [13]:
correct_apical_classification.append([curr_segment_id,curr_limb_idx])

In [ ]:
wrong_apical_classification.append([curr_segment_id,curr_limb_idx])